This the main notebook that is to be run by the user. First run all the cells and then run the last cell each time to search for a particular movie and get recommendation based on your selected movie.



NOTE: YOU NEED TO MOUNT THE GOOGLE DRIVE BEFORE USING THIS NOTEBOOK


Install the word2number library

In [16]:
pip install word2number

In [35]:
#import all the necessary libraries for this notebook.

import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from word2number import w2n
from time import sleep
from IPython.display import clear_output

In [18]:
#import all the excel data files that are necessary for this notebook as dataframes.

df = pd.read_excel('/content/drive/MyDrive/ml-25m/movies.xlsx')   #reading the movies data file provided in the original dataset.
df1 = pd.read_excel('/content/drive/MyDrive/ml-25m/ratings.xlsx')   #reading the ratings data file provided in the original dataset.
sdsd= pd.read_excel("/content/drive/MyDrive/ml-25m/genome-scores.xlsx")   #reading the genome-scores data file provided in the original dataset.
asas = pd.read_excel("/content/drive/MyDrive/ml-25m/genome-tags.xlsx")    #reading the genome-tags data file provided in the original dataset.
df2 = pd.read_excel("/content/drive/MyDrive/ml-25m/Copy of genre_clusters.xlsx")    #reading the genre_clusters data file that has been preprocessed and contains the cluster group information of each movie of the given dataset.

In [22]:
#load all the pre-processed matrices processed by the singular value decomposition method (svd).
#The movies in the data set are divided into 9 clusters and each cluster has its unique svd matrix which are loaded below.
#This matrices are in the form of numpy arrays.

V1a = np.load('/content/drive/MyDrive/ml-25m/V_data/V1a.npy')
V1b = np.load('/content/drive/MyDrive/ml-25m/V_data/V1b.npy')
V1c = np.load('/content/drive/MyDrive/ml-25m/V_data/V1c.npy')
V2a = np.load('/content/drive/MyDrive/ml-25m/V_data/V2a.npy')
V2b = np.load('/content/drive/MyDrive/ml-25m/V_data/V2b.npy')
V2c = np.load('/content/drive/MyDrive/ml-25m/V_data/V2c.npy')
V3a = np.load('/content/drive/MyDrive/ml-25m/V_data/V3a.npy')
V3b = np.load('/content/drive/MyDrive/ml-25m/V_data/V3b.npy')
V3c = np.load('/content/drive/MyDrive/ml-25m/V_data/V3c.npy')

In [20]:
#Create list of movies present in each movie cluster.

lst1a=[]
for i in df2[df2['Clusters'] == '1a']['MovieId']:
  lst1a.append(i)                                 #creates list of all movies present in cluster group 1a. 
lst1b=[]
for i in df2[df2['Clusters'] == '1b']['MovieId']:
  lst1b.append(i)                                 #creates list of all movies present in cluster group 1b.                                         
lst1c=[]
for i in df2[df2['Clusters'] == '1c']['MovieId']:
  lst1c.append(i)                                 #creates list of all movies present in cluster group 1c.
lst2a=[]
for i in df2[df2['Clusters'] == '2a']['MovieId']:
  lst2a.append(i)                                 #creates list of all movies present in cluster group 2a.
lst2b=[]
for i in df2[df2['Clusters'] == '2b']['MovieId']:
  lst2b.append(i)                                 #creates list of all movies present in cluster group 2b.
lst2c=[]
for i in df2[df2['Clusters'] == '2c']['MovieId']:
  lst2c.append(i)                                 #creates list of all movies present in cluster group 2c.
lst3a=[]
for i in df2[df2['Clusters'] == '3a']['MovieId']:
  lst3a.append(i)                                 #creates list of all movies present in cluster group 3a.
lst3b=[]
for i in df2[df2['Clusters'] == '3b']['MovieId']:
  lst3b.append(i)                                 #creates list of all movies present in cluster group 3b.
lst3c=[]
for i in df2[df2['Clusters'] == '3c']['MovieId']:
  lst3c.append(i)                                 #creates list of all movies present in cluster group 3c.

In [29]:
#Build a function that provides a bunch of similar movies based on the input movie. 

def movie_recommend(id):          #the function takes only one argument which is the movie Id.
  lst = []
  for i in df2[df2['MovieId']==id].Clusters:
    cls = i                       #finding the cluster group of the input movie Id.
 
  if cls == '1a':                 #Finding the svd matrix corresponding to the cluster group (to which the movie Id belongs) for further processing.
    V = V1a
    lst = lst1a
  elif cls == '1b':
    V = V1b
    lst = lst1b
  elif cls == '1c':
    V = V1c
    lst = lst1c
  elif cls == '2a':
    V = V2a
    lst = lst2a
  elif cls == '2b':
    V = V2b
    lst = lst2b
  elif cls == '2c':
    V = V2c
    lst = lst2c
  elif cls == '3a':
    V = V3a
    lst = lst3a
  elif cls == '3b':
    V = V3b
    lst = lst3b
  elif cls == '3c':
    V = V3c
    lst = lst3c

  #The movie Ids are not uniform by numbering. Some movie Ids are missing in the original dataset.

  #So to make the numbering uniform, a dictionary is made where the key is the original movie Id and its value is
  #the new uniformly numbered movie Id.

  asdf = {}
  for i in range(len(lst)):
    asdf[lst[i]] = i

  #defining the get_key function which enables us to get the key from the value of a dictionary.

  def get_key(dict, val):           #the function takes dictionary name as the first argument and value of a particular key in that dictionary as the second argument.
    for key, value in dict.items():
         if val == value:
             return key             #the function returns the key corresponding to the input dictionary and the value.

  #defining the cosine similarity function to get the top similar movie by applying the cosine similarity between two arrays.
  def top_cosine_similarity(data, movie_id, top_n):  #the function takes the svd data matrix/array as the first argument, movie Id as the second argument and how many top movies to return as the third argument.
      index = asdf[movie_id]
      movie_row = data[index, :]
      magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data)) 
      similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
      sort_indexes = np.argsort(-similarity)
      return sort_indexes[:top_n]   #the function returns an array containing new movie Ids of the top n similar movies.

  #now we will use the top_cosine_similarity function and print the movies using the movie Id present in the array that is returned by the top_cosine_similarity func.

  #printing the details of the movie which is inputed.
  movie_id = id
  print('your movie:\n')
  for i in df[df.movieId==movie_id].title:
    print(i)
  for j in df[df.movieId==movie_id].genres:
    print(j + '\n')

  top_n = 20                                                        #defining the number of movies to be recommended.
  sliced_matrix = V.T                                               #defining the svd data matrix that is to be provided as an argument to the top_cosine_similarity function.
  indexes = top_cosine_similarity(sliced_matrix, movie_id, top_n)    #use the top_cosine_similarity function.
 
  #print recommended movies.
  if len(indexes) > 0:
    print('Movies You may like:\n')
    for i in indexes:
      for j in df[df.movieId==get_key(asdf,i)].title:
        print(j)
      for j in df[df.movieId==get_key(asdf,i)].genres:
        print(j + '\n')

In [24]:
#Build a function that returns movies that are relevant to the input tag.

tag_movie_dict = {}                   #building a dictionary that has tag id as the key and list of all the movies (whose relevance with the tag id is greater than or equal to 0.5) as the value.
sdsd1 = sdsd[sdsd["relevance"] >= 0.5]
for i in range(len(asas)):
    sdsd2 = sdsd1[sdsd1["tagId"]==i+1]
    sdsds2 = sdsd2.sort_values('relevance', ascending=False)
    tag_movie_dict[i+1] = list(sdsds2["movieId"])

  
def movie_tagbased(a, check):        #the function takes the input tag as first argument and check (either 0 or 1) as the second argument. The function will print anything only if the check is equal to 1, else it will only return the desired value. 
  try:
      b = int(asas[asas["tag"] == a]["tagId"])
      if len(tag_movie_dict[b]) <= 10:
        for i in range(len(tag_movie_dict[b])):
          c = list(df[df["movieId"] == tag_movie_dict[b][i]]["title"])
          for j in c:
            if check == 1:
              print(str(i+1) + " " + j)
        if check == 1:
          print('')
          print('enter the number in front of the movie to watch the movie')
          input1 = int(input())-1
          clear_output()             #clears the screen (NOTE: This code is IDE specific and is only valid for google colab notebook)
          movie_recommend(int((tag_movie_dict[b])[input1]))
      else:
        for i in range(len((tag_movie_dict[b])[:10])):
          c = list(df[df["movieId"] == (tag_movie_dict[b][:10])[i]]["title"])
          for j in c:
            if check == 1:
              print(str(i+1) + " " + j)
        if check == 1:
          print()
          print('enter m for more suggestions')
          print('or enter the number in front of the movie to watch the movie')
          input1 = input()
          if input1 == 'm':
            for i in range(len((tag_movie_dict[b])[10:])):
              c = list(df[df["movieId"] == (tag_movie_dict[b][10:])[i]]["title"])
              for j in c:
                print(str(i+1+10) + " " + j)
            print('enter the number in front of the movie to watch the movie')
            input2  = int(input())-1
            clear_output()          #clears the screen
            movie_recommend(int((tag_movie_dict[b])[input2]))            
          else:
              clear_output()        #clears the screen
              movie_recommend(int((tag_movie_dict[b])[int(input1)-1]))       
  except:
      return("no such movies found")

In [41]:
#defining the get_key function to get the key from the value.
def get_key(dict, val):                   #the function takes dictionary name as the first argument and value of a particular key in that dictionary as the second argument.
    for key, value in dict.items():
         if val == value:
             return key                   #the function returns the key corresponding to the input dictionary and the value.

#building a function that returns the movies based on the names seached in search box.
def movie_namebased(user_input, check):   #the function takes user input as the first argument and check as the second argument. The role of check is the same as it is in movie_tagbased function.
  movie_list = [user_input]

  for i in df["title"]:
      movie_list.append(i)

  #using the tensorflow tokenizer to tokenize each word in the movie data set.
  tokenizer = Tokenizer()                 
  tokenizer.fit_on_texts(movie_list)
  index = tokenizer.word_index
  sequences = tokenizer.texts_to_sequences(movie_list) #creating sequences of the tokens created.

  l = user_input.split()
  l1 = []
  l3 = []
  for k in l:
      l2 = []
      try:
          v = [k, str(w2n.word_to_num(k))]
      except:
          v = [k]
      for i in v:
          for j in range(len(sequences)):
              if j > 0:
                  if index[i] in sequences[j]:
                      s = ""
                      l2.append(movie_list[j])
                      for k in sequences[j]:
                          s = s + " " + get_key(index, k)
                      l1.append(s[1:][:len(s)-6])
      l3.append(l2)
  l4 = set.intersection(*[set(list) for list in l3])
  l5 = []
  for i in l3:
    for j in i:
      l5.append(j)
  if l5 == []:
    ret1 = "no searches found"
  else:
    ret1 = None
    if check == 1:
      if l4 != set():
        dict_m = {}
        for i in range(len(l4)):
          print(str(i+1) + ' ' + (list(l4))[i])
          dict_m[i+1] = (list(l4))[i]
        print("\n")
        jcount = 0
        if len(l5) != len(list(l4)):
          print('Similar Results:')
        for j in range(len(l5)):
          if l5[j] not in l4:
            jcount = jcount + 1
            print(str(i+jcount+1) + " " + l5[j])
            dict_m[i+jcount+1] = l5[j]
      if l4 == set():
        dict_m = {}
        jcount = 1
        print('Did you mean:')
        print(str(jcount) + " " + l5[0] + '\n')
        dict_m[jcount] = l5[0]
        if len(l5)>0:
          print('Similar Results:')
        for j in range(1,len(l5)):
          if l5[j] not in l4:
            jcount = jcount + 1
            print(str(jcount) + " " + l5[j])  
            dict_m[jcount] = l5[j]
  if check == 0:
    dict_m = None  
    return([ret1, dict_m])
  if check == 1:
    return([ret1, dict_m])  

In [43]:
#This is the final cell that will run everytime to search a movie or a tag.
#This cell will run all the previously defined functions and return list of movies based on the search of the user and recommend some movies similar to the search

#getting the user input.
userinput = input()
user_input = userinput.lower() #making the user input to lower case.

clear_output()   #clear the screen

#running previously defined functions for movie suggestions.
if movie_tagbased(user_input, 0) == "no such movies found":
  if (movie_namebased(user_input, 0))[0] == "no searches found":
    print("no searches found")
  else:
    resu = movie_namebased(user_input, 1)
    print('\nenter the number in front of the movie to watch the movie')
    use_in = int(input())
    clear_output()
    for i in df[df['title'] == resu[1][use_in]]['movieId']:
      movie_recommend(i)
else:
  movie_tagbased(user_input, 1)

no searches found


THE ABOVE CELL IS THE FINAL CELL. RUN IT EVERYTIME TO SEARCH AND MOVIE AND GET RECOMMENDATIONS BASED ON YOUR MOVIE.